In [1]:
import numpy as np
from pymongo import MongoClient
from pyeasyga import pyeasyga

import time

Importing Pokemon Data:

In [2]:
client = MongoClient()
db = client['genetic_algo'] 

# Pokemon's teams collection
collection_poke_teams = db['pokemon_teams']
pokemon_teams = list(collection_poke_teams.find())

# Individual pokemon's collection
collec_indiv_pokemon = db['pokemons']
pokemons = list(collec_indiv_pokemon.find())


Defining some important functions:

In [3]:
def get_pokemon_individually():
    
    client = MongoClient()
    db = client['genetic_algo'] 

    # Individual pokemon's collection
    collec_indiv_pokemon = db['pokemons']
    pokemons = list(collec_indiv_pokemon.find())
    
    return pokemons


def get_pokemon_teams():
    
    client = MongoClient()
    db = client['genetic_algo'] 

    # pokemon's teams collection
    collec_poke_teams = db['pokemon_teams']
    collec_poke_teams = list(collec_poke_teams.find())
    
    for poke_team in collec_poke_teams:
        del poke_team['_id']    
        
    return collec_poke_teams

    
def best_against(pokeA, pokeB):
    
    best = 0
    pokemons_max_types = ['primaryType', 'secondaryType']
    
    for poke_typeA in pokemons_max_types:
        if poke_typeA in pokeA:

            for poke_typeB in pokemons_max_types:
                if poke_typeB in pokeB:
                    typeA_against_typeB = f'{poke_typeA}_against_{pokeB[poke_typeB]}'
                    if typeA_against_typeB in pokeA and pokeA[typeA_against_typeB] > best:

                        best = pokeA[typeA_against_typeB]
    return best
    

def battle(poke1, poke2):
    #print(poke1)
    against1 = best_against(poke1, poke2)
    against2 = best_against(poke2, poke1)

    return poke1['CP'] * against1 - poke2['CP'] * against2


def fitness(individual, data):
    # Starts the fit value with 0
    fit = 0

    rival_team = data
    
    if len(individual) == 3:  # Check if 3 pokemons were selected

        # Cada Pokémon do time do indivíduo batalhará contra todos os 3 do time rival
        for home_pokemon in individual:
            for rival_pokemon in rival_team:
                fit += battle(home_pokemon, rival_pokemon)

        #print(fit)
    return fit




def create_individual(data):

    pokemons = get_pokemon_individually()
    
    
    # Random selector 
    rnd_selector = np.random.default_rng()

    poke_rival_tm_idx = rnd_selector.integers(low = 0, high = len(pokemons), size = 3)
    poke_rival_tm = [pokemons[poke_idx] for poke_idx in poke_rival_tm_idx]


    return poke_rival_tm



# define and set the GA's mutation operation
def mutate(individual):

    client = MongoClient()
    db = client['genetic_algo'] 

    # Individual pokemon's collection
    collec_indiv_pokemon = db['pokemons']
    pokemons = list(collec_indiv_pokemon.find())

    # Random selector 
    rnd_selector = np.random.default_rng()

    mutate_index = rnd_selector.integers(low = 0, high = len(individual))
    new_pokemon_idx = rnd_selector.integers(low = 0, high = len(pokemons))
    individual[mutate_index] = pokemons[new_pokemon_idx]



# define and set the GA's mutation operation
def crossover(parent_1, parent_2):

    # Random selector 
    rnd_selector = np.random.default_rng()

    cut_point = rnd_selector.integers(low = 1, high = 3)

    child_1 = parent_1[:cut_point]
    child_1.extend(parent_2[cut_point:])

    child_2 = parent_2[:cut_point]
    child_2.extend(parent_1[cut_point:])

    return child_1, child_2



### Training the GA

Codigo abaix que mostra o fitness de cada individuo. Tem 1000 observacoes, cada 20 correponde a 1 geracao. Para pegar o de melhor geracao, armazenar em um vetor

In [4]:
""" rival_team = poke_rival_teams[0]['Team 0']
ga = pyeasyga.GeneticAlgorithm(
        rival_team,  
        population_size=20,
        generations=50,
        crossover_probability=0.8,
        mutation_probability=0.2,
        elitism=True
    )
ga.fitness_function = fitness
ga.crossover_function = crossover
ga.create_individual = create_individual

ga.mutate_function = mutate

ga.run() """

" rival_team = poke_rival_teams[0]['Team 0']\nga = pyeasyga.GeneticAlgorithm(\n        rival_team,  \n        population_size=20,\n        generations=50,\n        crossover_probability=0.8,\n        mutation_probability=0.2,\n        elitism=True\n    )\nga.fitness_function = fitness\nga.crossover_function = crossover\nga.create_individual = create_individual\n\nga.mutate_function = mutate\n\nga.run() "

In [11]:
poke_rival_teams = get_pokemon_teams()

for rival_team_idx, rival_team in enumerate(poke_rival_teams):
    ga = pyeasyga.GeneticAlgorithm(
        rival_team[f'Team {rival_team_idx}'],  
        population_size=20,
        generations=50,
        crossover_probability=0.8,
        mutation_probability=0.2,
        elitism=True
    )
    ga.fitness_function = fitness
    ga.crossover_function = crossover
    ga.create_individual = create_individual

    ga.mutate_function = mutate

    fitness_results  = dict()
    execution_results = dict()
    for i in range(5):
        start_time = time.time()
        ga.run()
        end_time = time.time()
        
        # Calculates the total execution time for each run
        execution_time = end_time - start_time 
        execution_results[f'Execution {i}'] = execution_time

        fitness_results[f'Execution {i}'] = ga.best_individual()
    
    # Saving the result
    
    rival_team['fitness_results'] = fitness_results
    rival_team['execution_results'] = execution_results

    time.sleep(5)

    

{'Team 0': [{'_id': ObjectId('66d60b4953343f07c4a35a04'), 'name': 'Marill', 'poke_id': '0182', 'stamina': 172, 'attack': 37, 'defense': 93, 'primaryType': 'Water', 'primaryType_against_Bug': 1, 'primaryType_against_Dark': 1, 'primaryType_against_Dragon': 1, 'primaryType_against_Electric': 2, 'primaryType_against_Fairy': 1, 'primaryType_against_Fighting': 1, 'primaryType_against_Fire': 0.5, 'primaryType_against_Flying': 1, 'primaryType_against_Ghost': 1, 'primaryType_against_Grass': 2, 'primaryType_against_Ground': 1, 'primaryType_against_Ice': 0.5, 'primaryType_against_Normal': 1, 'primaryType_against_Poison': 1, 'primaryType_against_Psychic': 1, 'primaryType_against_Rock': 1, 'primaryType_against_Steel': 0.5, 'primaryType_against_Water': 0.5, 'secondaryType': 'Fairy', 'secondaryType_against_Bug': 0.5, 'secondaryType_against_Dark': 0.5, 'secondaryType_against_Dragon': 0, 'secondaryType_against_Electric': 1, 'secondaryType_against_Fairy': 0.5, 'secondaryType_against_Fighting': 0.5, 'sec

In [42]:
best_fitness_per_generation = []
# Função para executar o algoritmo e salvar o melhor fitness de cada geração
def run_ga_with_fitness_tracking():
    for generation in range(ga.generations):
        ga.create_next_generation()

        # Captura o melhor indivíduo (aquele com o maior fitness) nesta geração
        best_individual = max(ga.current_generation, key=lambda x: x.fitness)
        
        # Salva o melhor fitness dessa geração
        best_fitness_per_generation.append(best_individual.fitness)

    # Opcional: imprima o melhor fitness de cada geração
    print("Melhor fitness por geração:")
    for gen_idx, fitness_value in enumerate(best_fitness_per_generation):
        print(f"Geração {gen_idx + 1}: {fitness_value}")

# Executando o algoritmo genético com rastreamento de fitness
run_ga_with_fitness_tracking()

Melhor fitness por geração:
Geração 1: 46672.5
Geração 2: 46672.5
Geração 3: 46672.5
Geração 4: 46672.5
Geração 5: 46672.5
Geração 6: 46672.5
Geração 7: 46672.5
Geração 8: 46672.5
Geração 9: 46672.5
Geração 10: 46672.5
Geração 11: 46672.5
Geração 12: 46672.5
Geração 13: 46672.5
Geração 14: 46672.5
Geração 15: 46672.5
Geração 16: 46672.5
Geração 17: 46672.5
Geração 18: 46672.5
Geração 19: 46672.5
Geração 20: 46672.5
Geração 21: 46672.5
Geração 22: 46672.5
Geração 23: 46672.5
Geração 24: 46672.5
Geração 25: 46672.5
Geração 26: 46672.5
Geração 27: 46672.5
Geração 28: 46672.5
Geração 29: 46672.5
Geração 30: 46672.5
Geração 31: 46672.5
Geração 32: 46672.5
Geração 33: 46672.5
Geração 34: 46672.5
Geração 35: 46672.5
Geração 36: 46672.5
Geração 37: 46672.5
Geração 38: 46672.5
Geração 39: 46672.5
Geração 40: 46672.5
Geração 41: 46672.5
Geração 42: 46672.5
Geração 43: 46672.5
Geração 44: 46672.5
Geração 45: 46672.5
Geração 46: 46672.5
Geração 47: 46672.5
Geração 48: 46672.5
Geração 49: 46672.5
G

In [13]:


data = get_pokemon_individually()


ga = pyeasyga.GeneticAlgorithm(
    data,  # Gera uma população inicial
    population_size=20,
    generations=50,
    crossover_probability=0.8,
    mutation_probability=0.2,
    elitism=True
)
ga.fitness_function = fitness
ga.crossover_function = crossover
ga.create_individual = create_individual

ga.mutate_function = mutate


# Executar o algoritmo
ga.run()
